# 3 Theory
In this section, the theory behind each of the used concepts and algorithms will be explained with their relevant equations. We will first explain the concept behind cross-validation and how it applies to our data. Then we will describe the 5 algorithms, starting with the four approaches classified as 'naive' because they are relatively easy and quick to calculate and implement, and because they work surprisingly well. These naive approaches however cannot be used for the normalization of ratings and the cumulative improvement of the RMSE, so we will also evaluate two other approaches. 

## 3.1 Cross-validation
To make sure that our results are reliable, we are interested in the accuracy of our model on data that was not used in the training process, and we will apply the 5-fold cross-validation scheme to achieve this. The whole data set is randomly split into 5 parts of more or less equal sizes, and we will then develop 5 models for each combination of 4 our of 5 parts. Each of the 5 parts is therefore used once as a test set, while the model is trained on the remaining 4 sets. Each of the obtained models is then tested to the test set that was not used to train the model. This way we generated 5 different estimates of the model accuracy, and their average is considered to be a good estimate of the error on the future data. <br>
In this assignment, our training sets consist of ~800.000 ratings, and we will test the created model on the remaining ~200.000 ratings (test set). This was done 5 times to the completely unique test sets (this means that if all 5 test sets are combined, we obtain the total data set consisting of ~1.000.000 ratings again).

## 3.2 Naive Approaches
The following formulas that represent the 5 naive approaches were taken from slide 17.
### 3.2.1 Naive Approach 1:Global Average Rating
$\;\;\;\;\;\;$ $X\approx U\cdot M$

## 3.4 Matrix factorization

The idea behind matrix factorization is similar to that of UV matrix decomposition discussed earlier. We will follow the approach taken in the paper [$\textit{gravity-Tikk.pdf}$](https://www.cs.uic.edu/~liub/KDD-cup-2007/proceedings/gravity-Tikk.pdf) (2007). Here our goal is also to approximate the matrix with ratings, now called X, as the product of two matrices U and M:<br>
### $\;\;\;\;\;\;$ $X\approx U\cdot M$, $\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$(1)<br>
where U is an I x K and M is a K x J matrix, so that u$_{ik}$ and m$_{kj}$ can be treated as the kth feature of the ith user and the jth movie respectively. The major differences with the UV decomposition is that we update the matrices U and M differently, and that they are also initialized differently. <br>
<br>
We first initialize the two matrices U and M by filling them with values between 0 and 1. We then iterate over each known rating element of X and compute the training error on the (i, j)th example:<br>
### $\;\;\;\;\;\;$ e$_{ij}$ = x$_{ij}$ - $ \sum_{k=1}^{K} u_{ik}m_{kj}  $, $\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$(2)<br>
where e$_{ij}$ is the training error on the (i, j)th example and x$_{ij}$ is an element in the matrix X. u$_{ij}$ and m$_{ij}$ denote the elements of U and M, so the sum in eq.2 denotes how the ith user would rate the jth movie according to the model. <br>
To minimize the RMSE, we have applied a simple gradient descent method to find a local minimum. The gradient of $e^2_{ij}$ is given by:<br>
### $\;\;\;\;\;\;$ $\frac{\partial}{\partial u_{ik}}e^2_{ij} = -2 e_{ij} \cdot m_{kj} $, $\frac{\partial}{\partial m_{kj}}e^2_{ij} = -2 e_{ij} \cdot u_{ik} $$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$(3)<br>
We then updated the weights in U and M in the opposite direction of the gradient to decrease the error, thereby better approximating x$_{ij}$. Furthermore, a regularization which prevents large weights is implemented to obtain a better prediction for unseen examples. The updated elements can now be calculated according to eq.3:<br>
### $\;\;\;\;\;\;$ u'$_{ik}$ = u$_{ik}$ + $\eta$(2e$_{ij}$m$_{kj}$ - $\lambda$u$_{ik}$$)$, $\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$(4)<br>
### $\;\;\;\;\;\;$ m'$_{kj}$ = m$_{kj}$ + $\eta$(2e$_{ij}$u$_{ik}$ - $\lambda$m$_{kj}$$)$, $\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;$(5) <br>
where $\eta$ is the learning rate and $\lambda$ is the regularization.

We through each known element of X which is not in the probe subset and apply eq.6 & 7 to update U and M. After each loop the RMSE w.r.t. the probe subset is computed to see if it has improved (i.e. decreased) from the previous iteration. For this assignment, we used a dimension of K=10 for the matrices U and M, a regularization of r=0.05, a learling rate of l=0.005 and a stop at 75 iterations through the loop. If the RMSE did not improve during two iterations, the loop is also terminated. 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import math
import sys

import datetime
import random

In [3]:
#Users data
usersData = pd.read_csv('users.dat', sep='::', header=None, names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], engine='python')

#Ratings data
ratingsData = pd.read_csv('ratings.dat', sep='::', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')
ratings_testData = pd.read_csv('ratings_test.dat', sep='::', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')

#Movies data
moviesData = pd.read_csv('movies.dat', sep='::', header=None, names=['MovieID', 'Title', 'Genres'], engine='python', encoding='latin1')

In [4]:
#Users data
print(usersData.head())

#Ratings data
print(ratingsData.head())
print(ratings_testData.head())

#Movies data
print(moviesData.head())

   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)         

In [5]:
def init_UM(X_train, K):
    """
    This function initializes two matrices U and V for the UV-decomposition
    """
    
    I, J = X_train.shape
    U_i = np.random.rand(I, K) #U is an I x K matrix with randomly distributed weights 
    M_i = np.random.rand(J, K) #M is an K x J matrix with randomly distributed weights 


    
    return U_i, M_i


In [6]:
def optimize_elements(X_train, K, num_iter, r, l, U, M):
    """
    Perform a single round of optimization for the U and M matrices using the equations from page 346.
    The first two loops for U and M is for looping through all elements in the matrix,
    and the loops over j/i and k represent the sums in the expressions for x and y. 
    """
    
    
    
    I, J = X_train.shape


    for i in range(I):
        for j in range(J):
            if X_train[i, j] >0: #i.e. non-NaN
                eij = X_train[i, j] - np.dot(U[i, :], M[j, :])
                #eij denotes the training error on the (i, j)th example

                for k in range(K):
                    #For all latent features, apply eq.6&7 in gravity-Tikk.pdf
                    U[i, k] = U[i, k] + l * (2 * eij * M[j, k] - r * U[i, k])
                    M[j, k] = M[j, k] + l * (2 * eij * U[i, k] - r * M[j, k])

    return U, M




In [ ]:
totalRMSE = 0
totalMAE = 0
K, num_iter, r, l = 10, 75, 0.05, 0.005

def converge_UM(X_train, X_test, U_i, M_i):
    """
    With this function we iterate upon the element optimization of U and V, stopping when either the RMSE
    falls below a chosen threshold or the improvement over the previous iteration becomes insignificant. 
    """
    count = 1 #count the number of iterations
    

    
    #initialize the errors of the previous and current step such that the condition is held for the first loop 
    rmse_old = float('inf')
    rmse_new = 10

    #U_i, M_i  = init_UM(X_train, K)
    
    while (rmse_old - rmse_new > 1e-5) and count <= num_iter:
        
        
        U,M = optimize_elements(X_train, K, num_iter, r, l, U_i, M_i)
        P = np.matmul(U, M.T)
        diff = X_test - P
        rmse_old = rmse_new
        rmse_new = np.sqrt(np.nanmean(diff**2)) #compute the Root-Mean Square Error between the two matrices M and P 

        if count_kfold==1:
            RMSE_values1.append(rmse_new)
        if count_kfold==2:
            RMSE_values2.append(rmse_new)
        if count_kfold==3:
            RMSE_values3.append(rmse_new)
        if count_kfold==4:
            RMSE_values4.append(rmse_new)
        if count_kfold==5:
            RMSE_values5.append(rmse_new)
        print('loop number {}'.format(count), "RMSE:", rmse_new)
        count += 1
        
    return U,M,P, rmse_new #return the final, updated U and V


RMSE_values1, RMSE_values2, RMSE_values3, RMSE_values4, RMSE_values5 = [], [], [], [], []

count_kfold = 1
kf = KFold(n_splits=5, shuffle=True, random_state=26)

for train_index, test_index in kf.split(ratingsData):
    print("\nFold number:", count_kfold)
    now = datetime.datetime.now() #To check how long it takes to run 
    print ("Time at start of fold number", count_kfold, " :", now.strftime("%Y-%m-%d %H:%M:%S"))   
    
    train_set = ratingsData.iloc[train_index]
    test_set = ratingsData.iloc[test_index]
    
    # Save the user and movie IDs from the training/test set of this fold, for data visualisation later! 
    userIDs = df_train.index.values #test and train IDs are equivalent due to the condition above
    movieIDs = df_train.columns.to_list()
    np.save("userID_fold"+str(count_kfold), userIDs)
    np.save("movieID_fold"+str(count_kfold), movieIDs)
    
    #First, determine the common set of movies and users in both sets
    common_users = np.intersect1d(train_set['UserID'], test_set['UserID'])
    common_movies = np.intersect1d(train_set['MovieID'], test_set['MovieID'])

    # Filter the data to include only the common movies and users
    train_set = train_set[(train_set['UserID'].isin(common_users)) & (train_set['MovieID'].isin(common_movies))]
    test_set = test_set[(test_set['UserID'].isin(common_users)) & (test_set['MovieID'].isin(common_movies))]

    # Convert data into a numpy array
    df_train = train_set.pivot(index="UserID", columns="MovieID", values="Rating")
    df_test = test_set.pivot(index="UserID", columns="MovieID", values="Rating")
    X_train = np.array(df_train.to_numpy())
    X_test = np.array(df_test.to_numpy())
    
    print("Training set:")
    print(X_train)
    print("Test set:")
    print(X_test)

    U_i, M_i = init_UM(X_train, K)
    U, M, P, rmse = converge_UM(X_train, X_test, U_i, M_i)
    print("Training matrix M:\n", X_train)
    print("Predicted matrix P:\n", P)

    print("MAE estimate of fold number ", count_kfold, ": ", rmse**2)
    print("RMSE estimate of fold number ", count_kfold, ": ", rmse)
    
    
    totalRMSE += rmse
    totalMAE += rmse**2
    

    np.save("predicted_U_"+str(count_kfold), U)
    np.save("predicted_M_"+str(count_kfold), M)
    count_kfold += 1
    

# Calculate the average RMSE over all folds
averageRMSE = totalRMSE / 5
averageMAE = totalMAE / 5

print("The Average MAE is", averageMAE)
print("The Average RMSE is", averageRMSE)
print("All RMSE values:\n")
print(RMSE_values1, RMSE_values2, RMSE_values3, RMSE_values4, RMSE_values5)


Fold number: 1
Time at start of fold number 1  : 2023-10-21 18:48:00
Training set:
[[ 5. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [ 3. nan nan ... nan nan nan]]
Test set:
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
loop number 1 RMSE: 1.0194382083749316
loop number 2 RMSE: 0.9379355836672594
loop number 3 RMSE: 0.9350160401266857
loop number 4 RMSE: 0.9319117418255155
loop number 5 RMSE: 0.9266869762004475
loop number 6 RMSE: 0.9202914870494888
loop number 7 RMSE: 0.9144582294462785
loop number 8 RMSE: 0.9095719369799069
loop number 9 RMSE: 0.9054251317118147
loop number 10 RMSE: 0.9018391333597123
loop number 11 RMSE: 0.8987171052596992
loop number 12 RMSE: 0.8960008497695284
loop number 13 RMSE: 0.8936397508688514
loop number